In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)

In [ ]:
current_env = "local"

if current_env == "local":
    data_path = "../ump-dataset"

elif current_env == "kaggle":
    data_path = "../input/ump-dataset"
    
elif current_env == "colab":
    pass

print("data_path:", data_path)

***
## loading data

In [ ]:
features = [f"f_{i}" for i in range(300)]

features = pd.read_parquet(f"{data_path}/train.parquet", columns=features)
target = pd.read_parquet(f"{data_path}/train.parquet", columns=["target",])
time = pd.read_parquet(f"{data_path}/train.parquet", columns=["time_id",])

In [ ]:
time_ids = np.sort(time.time_id.unique())
len(time_ids)

In [ ]:
n_time_steps = len(time_ids)
print("time steps:", n_time_steps)

valid_prop = 0.1
valid_size = int(0.1 * n_time_steps)
print("valid size:", valid_size)

In [ ]:
# train-valid splits
n_splits = 3
end_idx = n_time_steps 

splits = list()

for start_idx in np.arange(1211, 0, -valid_size)[1:n_splits+1]:
    valid_time_ids = time_ids[start_idx:end_idx]
    train_time_end = time_ids[start_idx]-1
    end_idx = start_idx
    
    train_idx = time.query("time_id <= @train_time_end").index
    valid_idx = time.query("time_id in @valid_time_ids").index
    splits.append((train_idx,valid_idx))

In [ ]:
import gc
gc.collect()

***
## model training: finding number of iterations

In [ ]:
def pearsonr(preds: np.array, dset: xgb.DMatrix):
    """
    Helper function to compute Pearson correlation 
    on validation dataset for LightGBM as tracking metric.
    Args:
        preds: 1d-array with the model predictions
        dset: DMatrix dataset with the labels
    Returs:
        Tuple with the corresponding output
    """
    labels = dset.get_label()
    return 'pearsonr', stats.pearsonr(preds, labels)[0]

In [ ]:
model_params = {
    "tree_method":"hist",
    "grow_policy":"depthwise",
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'eta': 0.05,
    'seed': 19,
    'verbosity': 1,
    'max_depth':7,
    'max_bin':256,
    'colsample_bytree':0.3,
    'subsample':0.9,
    'reg_alpha':1,
    'reg_lambda':0.1,
    'min_child_weight':1000,
}

In [ ]:
%%time

pretrain = True
metrics = {"corr_mean":list(), "corr_std":list(), "error_mean":list(), "error_std":list(),}

if pretrain:
    
    models = list()

    for train_idx,valid_idx in splits:
        
        # input datasets for xgb
        train_dset = xgb.DMatrix(
            data=features.loc[train_idx,:],
            label=target.loc[train_idx,"target"].values,
        )
        valid_dset = xgb.DMatrix(
            data=features.loc[valid_idx,:],
            label=target.loc[valid_idx,"target"].values,
        )
        
        model = xgb.train(
            params=model_params,
            num_boost_round=3000,
            dtrain=train_dset,
            evals=[(valid_dset,"valid"),],
            feval=pearsonr,
            maximize=True,
            early_stopping_rounds=50,
            verbose_eval=20,
        )
        models.append(model)

        xgb.plot_importance(model, importance_type="weight", max_num_features=30)
        xgb.plot_importance(model, importance_type="gain", max_num_features=30)
        plt.show()
        
        # residual analysis on oof predictions
        oof = target.loc[valid_idx,:].copy()
        oof["time_id"] = time.loc[valid_idx,"time_id"]
        oof["pred"] = model.predict(valid_dset)
        oof["target_abs"] = oof.eval("abs(target)")
        oof["dev"] = oof.eval("abs(target-pred)")

        corrs = oof.groupby("time_id").apply(lambda x: stats.pearsonr(x.target, x.pred)[0])
        corr_mean = corrs.mean()
        corr_std = corrs.std()
        error = oof.groupby("time_id").apply(lambda x: np.sqrt(np.mean((x.target-x.pred)**2)))
        error_mean = error.mean()
        error_std = error.std()
        
        metrics["corr_mean"].append(corr_mean)
        metrics["corr_std"].append(corr_std)
        metrics["error_mean"].append(error_mean)
        metrics["error_std"].append(error_std)

        plt.figure(figsize=(18,8))
        plt.subplot(1,2,1)
        corrs.plot()
        plt.axhline(
            y=corr_mean, 
            color='r', 
            linestyle='-', 
            label=f"corr_mean={corr_mean:.5f} & corr_std={corr_std:.5f}"
        )
        plt.grid()
        plt.ylabel("corr")
        plt.legend(loc="best")
        ##
        plt.subplot(1,2,2)
        error.plot()
        plt.axhline(
            y=error_mean, 
            color='r', 
            linestyle='-', 
            label=f"rmse_mean={error_mean:.5f} & error_std={error_std:.5f}"
        )
        plt.grid()
        plt.ylabel("rmse")
        plt.legend(loc="best")
        plt.show()

        plt.figure(figsize=(22,8))
        ##
        plt.subplot(1,3,1)
        plt.plot(oof.sort_values("target_abs").target_abs.values, oof.sort_values("target_abs").dev.values)
        plt.xlabel("target_abs")
        plt.ylabel("deviance (abs)")
        plt.grid()
        ##
        plt.subplot(1,3,2)
        plt.plot(oof.sort_values("target").target.values, oof.sort_values("target").dev.values)
        plt.xlabel("target")
        plt.ylabel("deviance (abs)")
        plt.grid()
        ##
        plt.subplot(1,3,3)
        plt.plot(oof.sort_values("target").target.values, oof.sort_values("target").pred.values)
        plt.xlabel("target")
        plt.ylabel("pred")
        plt.grid()
        plt.show()
        
        ## correlation vs target value
        oof["target_pct"] = ((100-1e-10)*oof.target.rank(pct=True)).astype(int)
        #oof["target_pct"] = 5*(oof.target_pct/5).astype(int)

        bucket_corr = (
            oof
            .groupby("target_pct")
            .apply(lambda x: stats.pearsonr(x.target, x.pred)[0])
            .reset_index()
            .rename({0:"corr"}, axis=1)
        )

        plt.figure(figsize=(12,5))
        plt.plot(bucket_corr["target_pct"], bucket_corr["corr"])
        plt.xlabel("target_quantile")
        plt.ylabel("correlation")
        plt.grid()
        plt.show()
           
    n_iterations = int(np.mean([m.best_iteration for m in models]))
    
else:
    # default value
    n_iterations = 600
    

print("n_iterations:", n_iterations)

In [ ]:
metrics

In [ ]:
# mean corr on validation
np.mean(metrics["corr_mean"])

In [ ]:
# mean rmse on validation
np.mean(metrics["error_mean"])

***
## model training

In [ ]:
seeds = [2,7,11,19,23]
models = list()

train_dset = xgb.DMatrix(
    data=features,
    label=target.target.values,
)

for seed in seeds:
    _model_params = dict(model_params)
    _model_params["seed"] = seed
    
    
    model = xgb.train(
        params=_model_params,
        num_boost_round=n_iterations,
        dtrain=train_dset,
        evals=[(train_dset,"train"),],
        feval=pearsonr,
        maximize=True,
        verbose_eval=50,
    )
    models.append(model)

    xgb.plot_importance(model, importance_type="weight", max_num_features=30)
    xgb.plot_importance(model, importance_type="gain", max_num_features=30)
    plt.show()

In [ ]:
for seed,model in zip(seeds,models): 
    model.save_model(f"../ump-artifacts/xgboost-gbrt/xgb-seed{seed}.json")

***
## inference

In [ ]:
if current_env == "kaggle":

    import ubiquant
    env = ubiquant.make_env()  
    iter_test = env.iter_test()
    
    features = [f"f_{i}" for i in range(300)]
    for (test_df, sample_prediction_df) in iter_test:  
        preds = [model.predict(test_df[features]) for model in models]
        sample_prediction_df['target'] = np.mean(preds, axis=0)
        env.predict(sample_prediction_df) 
        display(sample_prediction_df)

***